## Duplicate checks

In [3]:
import logging
import sys
import os

import pandas as pd
import numpy as np
from pandas import Timestamp
# pylint: disable=import-error disable=wrong-import-position
from pathlib import Path
sys.path.append(os.path.dirname(Path.cwd().parent))
print (os.path.dirname(Path.cwd().parent))
import pydit

logger=pydit.setup_logging_info()

/private/var/mobile/Containers/Shared/AppGroup/8649F1B4-9235-4351-9E08-DE43B9D56883/File Provider Storage/Repositories


AttributeError: module 'pydit' has no attribute 'setup_logging_info'

In [3]:
df=pd.read_excel("./input/data.xlsx")
df.columns

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
pydit.check_duplicates(df,["customer_id"],keep="first",indicator=True,return_non_duplicates=True)